# Refactor `dev_dataprep_train_add12.ipynb` into a python script to feed into data_fusion_pipeline

In [5]:
# import os
# os.makedirs('prep_src', exist_ok=True)
# !ipython kernel install --user --name vdsm_env_all_test2 --display-name "vdsm_env_all_test2"
# !conda init
# !conda activate /azureml-envs/azureml_46545bedc8821fd8b276345975dd73b7

Installed kernelspec vdsm_env_all_test2 in /home/azureuser/.local/share/jupyter/kernels/vdsm_env_all_test2


In [1]:
from platform import python_version
print(python_version())

3.6.9


In [5]:
%%writefile ./prep_src/vdsm_traindata_fusion.py
from fsspec.implementations.http import HTTPFileSystem
import pandas as pd
import math
import sys
import os
import time
import geopandas as gpd
from pyproj import Transformer
import pickle as pkl
import json 
import numpy as np
import rasterio
from rasterio.features import rasterize
import affine
import xarray as xr

from datetime import datetime
from functools import wraps

import argparse
from azureml.core.run import Run
from joblib import Parallel, delayed
# import space_blob
# from space_blob.user_blob import create_blob_stream
# from space_blob.user_blob import write_blob_client
# from space_blob.user_blob import get_blobnames

from vdsm_util import mean_value_dvar, get_mask_sh, get_previous_month_string
from vdsm_util import interpolate_single_band

def init():
    global columns, date_start, date_end, date_max_train, new_maxmon
    columns = ['year', 'month', 'evi', 'evi_prev', 'u10', 'u10_prev', 'v10', 'v10_prev', 't2m', 't2m_prev', 
            'lai_lv', 'lai_lv_prev', 'src', 'src_prev', 'ssr', 'ssr_prev', 'sp', 'sp_prev', 'e', 'e_prev', 
            'tp', 'tp_prev', 'tp_prev2', 'tp_prev3', 'swvl1', 'swvl1_prev', 'co2', 'co2_prev', 'lat_4326', 'lon_4326',
            'historic_mean']

    date_start = '2016-01'   # training month no earlier than this 
    date_end = '2021-07'   # training month no earlier than this 
    date_max_train = str((pd.period_range(date_end, date_end, freq='M')-12)[0])
    # train_year_min, train_year_max = int(date_start[0:4]), int(date_max_train[0:4])
    new_maxmon = int(date_max_train[-2:])   # cannot gather all training cols beyond train_year_max.new_maxmon


### Dump all the input requirements and keywords here first:
def fuse_all(s2_path, fc_path, co2_path, climate_path, mask_path,
                PV=False, NPV=True, 
                pixel_cls_url=None, random_select=-1, 
                create_new_fc=True, masked_use=True,
                **kwargs):
    """Fuse 4 categories of spatiotemporal data from the blob storage.

    Parameters
    ----------
    s2_path : str
        authenticated blob url path for Sentinel-2 data. e.g. 
        s2_path = create_blob_stream(blobname, t_expire=8)
    fc_path : str
        authenticated blob url path for fractional cover data.
    co2_path : str
        authenticated blob url path for co2 data.
    climate_path : str
        authenticated blob url path for climate data.
    mask_path : str
        authenticated blob url path for masks of the railway.
    PV: boolean
        True if selecting green veg pixels
    NPV: boolean
        True if selecting non-green veg pixels.
        If non of PV and NPV is true, return bare-land pixels.           
    pixel_cls_url: str, default = None
        authenticated blob url path for a pre-classified dataframe, 
            useful to reduce computation time if you already run the code once and
            saved the random_pv_df as a file in the blob.     
    random_select : int, default value = -1
        set to >0==N if you want to return N randomly sampled pixels in random_pv_df.
    create_new_fc : boolean, default True
        set this to False if pixel_cls_url is not None.
    masked_use : boolean, default=True
        set to False to disable the use of mask_path

    Returns
    -------
    ds2_all, ds_env: xarray 
    co2_df, random_pv_df: dataframes
        To be used in the next step of training data assembly 
         
    To-Do:
    link a logic between pixel_cls_url with create_new_fc
    """
    # PV = False   # select green veg
    # NPV = True   # select Non-green veg
    # create_new_fc = True   # classify pixels for the first time
    # random_select = -1    # if >0 return N random select pixels
    # Todo: if create_new_fc = false, then pixel_cls_uls must exist, also write out
    # masked_use = True  # use mask for the network or not


    # Read in co2 data
    ### To-Do: find a better way to write this so that the line number like 57 is auto-detected?
    ### Need to write a error check code in case co2 table format changes
    column_names = HTTPFileSystem().open(co2_path).readlines()[57][2:-1].decode().split()
    co2_df = pd.DataFrame(columns=column_names)

    with HTTPFileSystem().open(co2_path) as f:
        for line in f.readlines()[58:]:
            vals = line[2:-1].decode().split()
            dict_vals = {}
            for i in range(len(column_names)):
                dict_vals[column_names[i]] = vals[i]
            co2_df = co2_df.append(dict_vals, ignore_index=True) 
    
    co2_df['month-year'] = co2_df['year'].astype('str') + '-' + co2_df['month'].astype('str').str.rjust(2,'0')
    co2_df = co2_df.set_index(['month-year'])
    co2_df.drop(columns=['year', 'month', 'decimal', 'trend'], inplace=True)
    co2_df['average'] = co2_df['average'].astype('float')

    # Read in climate data
    ds_env = xr.open_dataset(HTTPFileSystem().open(climate_path))
    ds_env = ds_env.drop(['lai_hv'])   # cuz lai_hv column is problematic, mostly zeros

    # Read in S2 data (minimal time range: 2015-12 to 2021-07)
    with HTTPFileSystem().open(s2_path, 'rb') as f:
        ds2_all = pkl.load(f)
        # to do, write a if else here
        ds2_all['evi'] = 2.5*((ds2_all.nbart_nir_1 - ds2_all.nbart_red) / (ds2_all.nbart_nir_1 + 6*ds2_all.nbart_red - 7.5*ds2_all.nbart_blue +1))

    # Read in fractional cover data
    with HTTPFileSystem().open(fc_path, 'rb') as f:
        ds_fc = pkl.load(f)
    
    # Read in mask file
    ## need to check if this way of reading in the mask is alright
    gdf = gpd.read_file(HTTPFileSystem().open(mask_path))
    gdf_3577 = gdf.to_crs("EPSG:3577")
    mask = get_mask_sh(ds2_all, gdf_3577)
    ds_fc_m = ds_fc.where(mask)
    ds_s2_m = ds2_all.where(mask)

    # Conditions are set with error bars in mind. pv+ npv + bs != 100 cuz of error bars.
    if create_new_fc:
        ds_fc_time_median = ds_fc.median(dim='time', skipna=True)
        if PV:
            filt1 = ds_fc_time_median['PV_PC_90'] > ds_fc_time_median['NPV_PC_90']
            filt2 = ds_fc_time_median['PV_PC_90'] > ds_fc_time_median['BS_PC_90']
            ###  condtion for training
            cond_pv = (ds_fc_time_median['PV_PC_90'] >= 20) & (ds_fc_time_median['BS_PC_90'] < 60)
    #         cond_pv = (ds_fc_time_median['PV_PC_90'] > 20) & (ds_fc_time_median['BS_PC_90'] < 60)
    #         cond_pv = (filt1 & filt2)     # this results in few data
        elif NPV:
            filt1 = (ds_fc_time_median['NPV_PC_90'] >= 45) & (ds_fc_time_median['PV_PC_90'] < 20)
            filt2 =  ds_fc_time_median['BS_PC_90'] < 60
            ###  condtion for training
            cond_pv = (filt1 & filt2) 
        else:
            cond_pv = (ds_fc_time_median['NPV_PC_90'] < 45) & (ds_fc_time_median['BS_PC_90'] >= 60)

        if masked_use:
            mds_fc_time_median = ds_fc_m.median(dim='time', skipna=True)   ## for masked regions
            mcony = mds_fc_time_median['PV_PC_90'].where(cond_pv, drop=True).squeeze()
        else:        
            mcony = ds_fc_time_median['PV_PC_90'].where(cond_pv, drop=True).squeeze()

        mcony_df = mcony.to_dataframe().dropna()
        mcony_df.reset_index(inplace=True)
        mcony_df.drop_duplicates(subset=['y','x'], keep='first', inplace=True)
        transformer = Transformer.from_crs("EPSG:3577", "EPSG:4326")
        y_4326, x_4326 = transformer.transform(mcony_df.x.values, mcony_df.y.values)
        mcony_df['y_4326'] = y_4326
        mcony_df['x_4326'] = x_4326

        ### random samples
        if random_select > 0:
            random_pv_df = mcony_df.sample(n=random_select, replace=False)
        else:
            random_pv_df = mcony_df.copy()
    else:
        random_pv_df = pd.read_csv(pixel_cls_url)
        
    # if write_to_blob:
    #     blob_client = write_blob_client(file_pv_coords, parent_dir='odc_vm_data/odc_outputs/')
    #     blob_client.upload_blob(random_pv_df.to_csv(encoding='utf-8'), overwrite=True)

    # df_concat = []   # did you use this ?
    return ds2_all, ds_env, co2_df, random_pv_df


def paralle_pixel(s2_lat, s2_lon, s2_lat_4326, s2_lon_4326, 
                        ds2_all=None, ds_env=None, co2_df=None):
    """
    Purpose: create a dataframe with columns of features for training
    Parameters: 
    ------ 
    s2_lat, s2_lon: float, input coordinates of one pixel in CRS 3577
    s2_lat_4326, s2_lon_4326: float, input coordinates of one pixel in CRS4326
    ds2_all, ds_env: xarray, fused data from fuse_all()
    co2_df: dataframe, fused data from fuse_all()
    -----
    Return:
    A dataframe with rows representing a time, and columns representing features
    The function is parallelisable. 

    """

    pixel_df = pd.DataFrame(columns=columns)

    dataset_dict = {}

    month = str((pd.period_range(date_start, date_start, freq='M')-1)[0])
    dataset_dict[-1] = ds2_all.sel(x=s2_lon, y=s2_lat, time=month, method='nearest')  

    bad_index = [-999]
    count = 0
    for year in range(int(date_start[0:4]), int(date_end[0:4])+1):
        for month in range(12):
            if year == int(date_end[0:4]) and month >= int(date_end[-2:]):
                break
            else:
                date = (str(year)+'-'+str(month+1).rjust(2,'0'))
                dataset_dict[count] = ds2_all.sel(x=s2_lon, y=s2_lat, time=date, method='nearest')
                dataset_dict[count] = dataset_dict[count].drop_vars(['nbart_red', 'nbart_blue', 'nbart_nir_1'])
                if len(dataset_dict[count].evi.data) == 0:
                    bad_index.append(count)
                    print(f'Warning!!! Year-Month {date} has no data, count is {count}!!!')
                count = count + 1

    for bad_index in bad_index:
        if bad_index > -1:
            dataset_dict[bad_index] = dataset_dict[bad_index+1]   ### fix the warning
    # print(len(dataset_dict))

    count = 0
    for year in range(int(date_start[0:4]), int(date_max_train[0:4])+1):
        for month in range(12):
            if year == int(date_max_train[0:4]) and month >= new_maxmon:
                break
            else:

                # getting the date into the string format of e.g. 2016-01, ...2016-11
                date = (str(year)+'-'+str(month+1).rjust(2,'0'))

                # this dictionary gets added as a row to the pixel_df dataframe
                pd_vars = {}

                # retrieving the actual EVI values for the 2, 4, and 6 month periodd 
                pd_vars['actual_2m'] = mean_value_dvar(dataset_dict[count+2], var='evi')
                pd_vars['actual_4m'] = mean_value_dvar(dataset_dict[count+4], var='evi')
                pd_vars['actual_6m'] = mean_value_dvar(dataset_dict[count+6], var='evi')
                pd_vars['actual_10m'] = mean_value_dvar(dataset_dict[count+10], var='evi')
                pd_vars['actual_12m'] = mean_value_dvar(dataset_dict[count+12], var='evi')
                # print(count+12)
    #             print('checking ------', str(dataset_dict[count+0].time[0].data))
                pd_vars['year'] = int(str(dataset_dict[count+0].time[0].data)[:4])
                pd_vars['month'] = int(str(dataset_dict[count+0].time[0].data)[5:7])

                #### TTY adding extra info here, should be outside of the loop
                pd_vars['lat_4326'] = s2_lat_4326
                pd_vars['lon_4326'] = s2_lon_4326

                month_year = str(pd_vars['year']) + '-' + str(pd_vars['month']).rjust(2, '0')
                previous_month_year = get_previous_month_string(month_year)

                pd_vars['evi_prev'] = mean_value_dvar(dataset_dict[count-1], var='evi')
                pd_vars['evi'] = mean_value_dvar(dataset_dict[count+0], var='evi')

                for band in ds_env.data_vars:
                    pd_vars[band] = interpolate_single_band(ds_env, s2_lat, s2_lon, band, month_year)
                    pd_vars[band+'_prev'] = interpolate_single_band(ds_env, s2_lat, s2_lon, band, previous_month_year)

                pd_vars['tp_prev2'] = interpolate_single_band(ds_env, s2_lat, s2_lon, 'tp', get_previous_month_string(previous_month_year))
                pd_vars['tp_prev3'] = interpolate_single_band(ds_env, s2_lat, s2_lon, 'tp', get_previous_month_string(get_previous_month_string(previous_month_year)))
                pd_vars['co2'] = co2_df.loc[month_year]['average']
                pd_vars['co2_prev'] = co2_df.loc[previous_month_year]['average']
                pixel_df = pixel_df.append(pd_vars, ignore_index=True)
                count = count + 1


    # this cell is for adding the seasonal change features
    # first we obtain all data points for the pixel of interest
    month_evi_dict = {}

    for i in range(1,13):
        month_evi_dict[i] = []

    time_index = 0
    for time in ds2_all.time.data:    # using 2015 dec -2020 dec data
        month_index = int(str(time)[5:7])
    #     evi_val = ds2_all.isel(time=time_index)['evi'].data[0][0]
        evi_val = ds2_all.sel(x=s2_lon, y=s2_lat, time=time, method='nearest')['evi'].data
        if evi_val < 1.5 and evi_val > -1.5:
            month_evi_dict[month_index].append(evi_val)
        time_index += 1

    # use the above dictionary to create a new dictionary mapping each month to the mean EVI    

    month_evi_means_dict = {}

    for i in range(1,13):
        month_evi_means_dict[i] = np.mean(np.array(month_evi_dict[i]))

    # this dictionary contains the EVI changes
    # for each month, we have a dictionary with the keys 2, 4, 6, 10, 12
    # representing the change in 2 months, 4 months, 6, 10, 12 months
    # e.g. evi_changes[1][4] returns the change in EVI from January to May (4 months in the re)

    evi_changes = {}

    for i in range(1,13):
        evi_changes[i] = {}
        for j in [2, 4, 6, 10]:
            # print(i, ((i+j-1)%12+1))
            evi_changes[i][j] = (month_evi_means_dict[(i+j-1)%12+1] - month_evi_means_dict[i])/month_evi_means_dict[i]

    seasonal_change = []
    for val in pixel_df['month']:
        seasonal_change.append(evi_changes[val][2])    
    pixel_df['seasonal_change_2m'] = seasonal_change

    seasonal_change = []
    for val in pixel_df['month']:
        seasonal_change.append(evi_changes[val][4])  
    pixel_df['seasonal_change_4m'] = seasonal_change

    seasonal_change = []
    for val in pixel_df['month']:
        seasonal_change.append(evi_changes[val][6])   
    pixel_df['seasonal_change_6m'] = seasonal_change

    seasonal_change = []
    for val in pixel_df['month']:
        seasonal_change.append(evi_changes[val][10])   
    pixel_df['seasonal_change_10m'] = seasonal_change


    historic_mean = []
    for val in pixel_df['month']:
        historic_mean.append(month_evi_means_dict[val])    
    pixel_df['historic_mean'] = historic_mean
    return pixel_df


def run_para(random_pv_df=None, n_jobs=4, ds2_all=None, ds_env=None, co2_df=None):
    """Parallel process dataframes across n_jobs
    random_pv_df, ds2_all, ds_env, co2_df: outputs from  fuse_all()
    n_jobs: number of jobs for parallel processing
    """
    results = Parallel(n_jobs=n_jobs)(delayed(paralle_pixel)(s2_lat, s2_lon, s2_lat_4326, s2_lon_4326, 
                              ds2_all=ds2_all, ds_env=ds_env, co2_df=co2_df) \
                    for s2_lat, s2_lon, s2_lat_4326, s2_lon_4326 \
                        in zip(random_pv_df.y, random_pv_df.x, random_pv_df.y_4326, random_pv_df.x_4326))
    return results    

 
def main():
    print("Running vdsm_data_fusion.py")
    run = Run.get_context()

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--s2-path",
        type=str,
        dest='s2_path',
        help="path of s2 data in pkl",
        default="s2.pkl",
    )
    parser.add_argument(
        "--fc-path",
        type=str,
        dest='fc_path',
        default='fc.pkl',
        help=("path of fractional cover data in pkl,\
               must be aligned with s2")
    )
    
    parser.add_argument(
        "--co2-path",
        type=str,
        dest='co2_path',
        default='co2_mm_gl_url.txt',
        help=("path of co2 data in txt,\
               read in as a df")
    )

    parser.add_argument(
        "--climate-path",
        type=str,
        dest='climate_path',
        default='era-mon-2015-2021.nc',
        help=("path of climate data in netcdf,\
               read in as an xarray")
    )

    parser.add_argument(
        "--mask-path",
        type=str,
        dest='mask_path',
        default='mask.shp',
        help=("path of rail masks,\
               can be geojson or shp files")
    )

    parser.add_argument(
        "--PV",
        type=bool,
        dest='PV',
        default=False,
        help=("Set true for green veg,\
               if bare land, set False for both PV, NPV")
    )

    parser.add_argument(
        "--NPV",
        type=bool,
        dest='NPV',
        default=True,
        help=("Set true for non-green veg,\
               if bare land, set False for both PV, NPV")
    )

    parser.add_argument(
        "--random-n",
        type=int,
        dest='random_n',
        default=50,
        help=("number of pixels to randomly select for training,\
               if <0, use all pixels")
    )

    parser.add_argument(
        '--fused-data-train-folder', 
        type=str, dest='fused_data_train_folder', 
        default='fused_data_train', 
        help='Folder for fused training data results'
    )

    parser.add_argument(
        '--fused-data-train-filename', 
        type=str,
        dest='fused_data_train_filename',
        default='fusetrain_data.csv', 
        help='Filename for fused training data results')

    args = parser.parse_args()
    s2_path = args.s2_path
    fc_path = args.fc_path
    co2_path = args.co2_path
    climate_path = args.climate_path
    mask_path = args.mask_path
    PV = args.PV
    NPV = args.NPV
    random_n = args.random_n
    save_folder = args.fused_data_train_folder
    save_file = args.fused_data_train_filename

    ds2_all, ds_env, co2_df, random_pv_df = fuse_all(s2_path, fc_path, co2_path,
                                     climate_path, mask_path, PV=False, NPV=True)
    init()
    if random_n > 0:
        random_pv_df_train = random_pv_df.sample(n=random_n, replace=False)
    else:
        random_pv_df_train = random_pv_df.copy()

    results = run_para(random_pv_df_train,  ds2_all=ds2_all, ds_env=ds_env, co2_df=co2_df)
    df_all_pv = pd.concat(results)

    ## how to save this dataframe? 
    # Save the prepped data
    print("Saving Data...")
    os.makedirs(save_folder, exist_ok=True)
    save_path = os.path.join(save_folder, save_file)
    df_all_pv.to_csv(save_path, header=True)
    run.tag("run_type", value="fuse_data_for_training")
    print(f"tags now present for run: {run.tags}")
    run.complete()

if __name__ == '__main__':
    main()

Overwriting ./prep_src/vdsm_traindata_fusion.py


In [5]:
# ### We only need to do this one time
# # query_s2_all = True         # set to true if loading data directly from DEA
# # write_to_local = False    # set to true if writing the pkl file to the current folder
# write_to_blob = False    # set to true if writing the pkl file to a blob location

# ### coodinate bounds, must be in CRS of 4326 
# max_lon_4326 = 118.760256
# min_lon_4326 = 118.73415
# max_lat_4326 = -22.940174
# min_lat_4326 = -22.979836

# lon_range = (min_lon_4326, max_lon_4326)
# lat_range = (min_lat_4326, max_lat_4326)
# bbox = [min_lon_4326, min_lat_4326, max_lon_4326, max_lat_4326]

In [18]:
# df_all_pv = pd.concat(results)
# write_to_local = False
# write_to_blob = True

# if write_to_local:
#     df_all_pv.to_csv(train_file_csv)
#     df_all_pv.to_pickle(train_file_pkl)


# if write_to_blob:
#     t1 = time.time()
#     blob_client = write_blob_client(train_file_pkl, parent_dir='odc_vm_data/odc_outputs/training_data/')
#     blob_client.upload_blob(pkl.dumps(df_all_pv), overwrite=True)
    
#     blob_client = write_blob_client(train_file_csv, parent_dir='odc_vm_data/odc_outputs/training_data/')
#     blob_client.upload_blob(df_all_pv.to_csv(encoding='utf-8'), overwrite=True)

#     t2 = time.time()
#     print('Uploading pkl to blob time {:.2f}mins'.format((t2 - t1)/60))

Uploading pkl to blob time 0.00mins


In [19]:
# blob_cont = get_blobnames('')
# file_del = 'odc_vm_data/odc_outputs/training_dataW_angelas_npv_allpx_monthly_train_test.pkl'
# try:
#         blob_cont.delete_blob(file_del)
# except:
#     print('no such file')   

/home/azureuser/cloudfiles/code/Users/Tiantian.Yuan/test_only/era-mon-2015-2021.nc
/mnt/batch/tasks/shared/LS_root/mounts/clusters/ty-test-oct8/code/Users/Tiantian.Yuan/data_tmp/mim_data_optical_water_stdev.csv
/mnt/batch/tasks/shared/LS_root/mounts/clusters/ty-test-oct8/code/Users/Tiantian.Yuan/data_tmp/mim_models_data.csv
/mnt/batch/tasks/shared/LS_root/mounts/clusters/ty-test-oct8/code/Users/Tiantian.Yuan/data_tmp/rain_simple.csv
/mnt/batch/tasks/shared/LS_root/mounts/clusters/ty-test-oct8/code/Users/Tiantian.Yuan/data_tmp/rain_simple_max.csv
HC_pred_green_from2021May_v1.tif
HC_pv_100px_monthly_train.csv
Mesa_pv_100px_monthly_train.csv
data/32a801906097225af35a5848adb124ba/response.tiff
gpr_updated.csv
hello.txt
master-auto_geometry.csv
mim_models_data.csv
model_features.csv
odc_vm_data/odc_outputs/W_angelas_2_evibands_2015-12_2018-12_test.pkl
odc_vm_data/odc_outputs/W_angelas_2_evibands_2015-12_2021-10_test.pkl
odc_vm_data/odc_outputs/W_angelas_2_evibands_2019-01_2021-10_test.pkl
o